# Intro about project in the notebook

#### A startup called Sparkify wants to analyze the data they've been collecting on songs and user activity on their new music streaming app. The analysis team is particularly interested in understanding what songs users are listening to. Currently, there is no easy way to query the data to generate the results, since the data reside in a directory of CSV files on user activity on the app.

#### They'd like a data engineer to create an Apache Cassandra database which can create queries on song play data to answer the questions, and wish to bring you on the project. Your role is to create a database for this analysis. You'll be able to test your database by running queries given to you by the analytics team from Sparkify to create the results.

 # Part 1. ETL Pipeline for Pre-Processing the Files

#### 1.1 Import Python packages 

In [10]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### 1.2 Creating list of filepaths to process original event csv data files

In [11]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
   

/home/workspace


#### 1.3 Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [12]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


#### 1.4 check the number of rows in your csv file


In [13]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part 2. Apache Cassandra

#### For this project I work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory. 
 #### The file contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> <br>

<img src="images/image_event_datafile_new.jpg">

### Working with Apache Cassandra

#### 2.1 Creating a Cluster

In [22]:
# Define a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need definied a session
session = cluster.connect()

#### 2.2 Create Keyspace

In [23]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project2 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### 2.3 Set Keyspace

In [24]:
try:
    session.set_keyspace('project2')
except Exception as e:
    print(e)

## Follow I show different querys by answer a business questions 



### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4







#### CREATE Table session_item

In [25]:
query1 = "CREATE TABLE IF NOT EXISTS session_item"
query1 = query1 + "(sessionId text, itemInSession text, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)                  

#### INSERT data into Table session_item

In [26]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements into the `query` variable
        query = "INSERT INTO session_item (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
         session.execute(query, (line[8], line[3], line[0], line[9], float(line[5])))

IndentationError: unexpected indent (<ipython-input-26-4b81ddfd7554>, line 10)

#### SELECT to verify that the data have been inserted into each table

In [28]:
query = "select artist,song,length from session_item WHERE sessionId = '338' and itemInSession = '4'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
     print (row)

##  Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### CREATE table user_session

In [ ]:
query2 = "CREATE TABLE IF NOT EXISTS user_session"
query2 = query2 + "(userId text, sessionId text, itemInSession text, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId),itemInSession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)  

                    

#### INSERT data into Table user_session

In [21]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (userId, sessionId, itemInSession, artist, song,firstName,lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s,%s,%s)"
        session.execute(query, (line[10], line[8], line[3], line[0], line[9], line[1],line[4]))

#### SELECT to verify that the data have been inserted into each table

In [22]:
select2 = "select artist,song,itemInSession,song, firstName, lastName from user_session\
            WHERE userId = '10' and sessionId = '182'"

try:
    rows = session.execute(select2)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", iteminsession='0', song_="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', iteminsession='1', song_='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', iteminsession='2', song_='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', iteminsession='3', song_='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### CREATE table song

In [23]:
query3 = "CREATE TABLE IF NOT EXISTS song"
query3 = query3 + "(song text, userId text, firstName text, lastName text,\
                    PRIMARY KEY (song, userId))"
try:
    session.execute(query3)
except Exception as e:
    print(e)                   

#### INSERT tabla into table song

In [24]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song (song,userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[10], line[1],line[4]))

#### SELECT to verify that the data have been inserted into each table

In [25]:
select3 = "select song, firstName, lastName from song WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(select3)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(song='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(song='All Hands Against His Own', firstname='Tegan', lastname='Levine')
Row(song='All Hands Against His Own', firstname='Sara', lastname='Johnson')


## Important: 

### Drop the tables before closing out the sessions

In [26]:
query = "drop table session_item"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()